# 🤖 Multi-Agent Task Allocation with RL — Real LLM Calls

This notebook demonstrates a **multi-agent system** where agents with specialized roles process tasks, with **Reinforcement Learning (Q-Learning)** guiding optimal task allocation — all backed by **real LLM API calls** via **LangChain/LangGraph**.

## Architecture

```
┌──────────────────────────────────────────────────────────────────────────────┐
│            MULTI-AGENT RL TASK ALLOCATION (Real LLM Calls)                   │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  ┌─────────┐    ┌──────────────────┐    ┌──────────────────┐                │
│  │ INCOMING │───►│  RL COORDINATOR   │───►│  AGENT POOL      │                │
│  │   TASK   │    │  (Q-Learning)     │    │                  │                │
│  └─────────┘    │                   │    │ 🔬 Researcher    │                │
│                 │ • State: task type │    │ 📊 Analyst       │                │
│                 │ • Action: assign   │    │ 💻 Coder         │                │
│                 │ • Reward: quality  │    │ ✅ Validator     │                │
│                 └────────┬──────────┘    └────────┬─────────┘                │
│                          │                        │                          │
│                          │    LangGraph            │  Real LLM Calls         │
│                          │    Orchestration        │  (OpenAI / Anthropic)   │
│                          ▼                        ▼                          │
│                 ┌──────────────────┐    ┌──────────────────┐                │
│                 │  LLM-AS-JUDGE    │◄───│    AGENT OUTPUT   │                │
│                 │                  │    │                  │                │
│                 │ • Quality Score  │    │ • Research docs  │                │
│                 │ • Relevance      │    │ • Analysis       │                │
│                 │ • Completeness   │    │ • Code           │                │
│                 │ • Agent Match    │    │ • Validation     │                │
│                 └────────┬─────────┘    └──────────────────┘                │
│                          │                                                  │
│                          ▼                                                  │
│                 ┌────────────────────────────────────────────┐              │
│                 │  RL UPDATE: Learn optimal task routing      │              │
│                 └────────────────────────────────────────────┘              │
└──────────────────────────────────────────────────────────────────────────────┘
```

## Key Differences from Simulated Version

| Aspect | Simulated | This Notebook |
|--------|-----------|---------------|
| **Agent Processing** | Mock tools returning fake data | Real LLM calls via LangChain |
| **Task Execution** | Random success simulation | Actual LLM-generated outputs |
| **Quality Evaluation** | Heuristic scores | LLM-as-Judge evaluation |
| **Workflow Orchestration** | Manual loops | LangGraph state machine |
| **RL Algorithm** | PPO (impractical with API calls) | Q-Learning (practical, sample-efficient) |
| **Cost Tracking** | None | Real token usage and USD costs |

## Prerequisites

- `OPENAI_API_KEY` environment variable set
- `ANTHROPIC_API_KEY` environment variable set

## 1. Environment Setup

In [ ]:
%pip install -q numpy pandas matplotlib seaborn \
    langchain langchain-openai langchain-anthropic langgraph \
    langchain-core pydantic tiktoken

In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Any, TypedDict
from dataclasses import dataclass
from enum import Enum
from collections import deque
import random
import logging
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field

# LangGraph imports
from langgraph.graph import StateGraph, END, START

# Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger(__name__)

np.random.seed(42)
random.seed(42)

print("✅ All imports successful!")

In [ ]:
assert os.environ.get("OPENAI_API_KEY"), "❌ OPENAI_API_KEY not set!"
assert os.environ.get("ANTHROPIC_API_KEY"), "❌ ANTHROPIC_API_KEY not set!"
print("✅ API keys verified")
print(f"   OpenAI key:    ...{os.environ['OPENAI_API_KEY'][-6:]}")
print(f"   Anthropic key: ...{os.environ['ANTHROPIC_API_KEY'][-6:]}")

## 2. LLM Service Layer

Wraps OpenAI and Anthropic with token tracking and cost estimation.

In [ ]:
class LLMProvider(Enum):
    OPENAI = "openai"
    ANTHROPIC = "anthropic"


@dataclass
class LLMResponse:
    content: str
    provider: str
    model: str
    input_tokens: int
    output_tokens: int
    total_tokens: int
    latency_seconds: float
    cost_estimate: float


class LLMService:
    """Manages LLM calls with token & cost tracking."""

    PRICING = {
        "gpt-4o-mini": {"input": 0.15, "output": 0.60},
        "gpt-4o": {"input": 2.50, "output": 10.00},
        "claude-3-5-haiku-latest": {"input": 0.80, "output": 4.00},
        "claude-3-5-sonnet-latest": {"input": 3.00, "output": 15.00},
    }

    def __init__(
        self,
        openai_model: str = "gpt-4o-mini",
        anthropic_model: str = "claude-3-5-haiku-latest",
        temperature: float = 0.7,
        max_tokens: int = 1024,
    ):
        self.openai_model_name = openai_model
        self.anthropic_model_name = anthropic_model
        self.openai_llm = ChatOpenAI(model=openai_model, temperature=temperature, max_tokens=max_tokens)
        self.anthropic_llm = ChatAnthropic(model=anthropic_model, temperature=temperature, max_tokens=max_tokens)
        self.total_cost = 0.0
        self.call_count = 0
        self.call_log: List[Dict[str, Any]] = []

    def _estimate_cost(self, model: str, inp: int, out: int) -> float:
        p = self.PRICING.get(model, {"input": 1.0, "output": 3.0})
        return (inp * p["input"] + out * p["output"]) / 1_000_000

    def call(self, system_prompt: str, user_message: str,
             provider: LLMProvider = LLMProvider.OPENAI) -> LLMResponse:
        msgs = [SystemMessage(content=system_prompt), HumanMessage(content=user_message)]
        llm = self.openai_llm if provider == LLMProvider.OPENAI else self.anthropic_llm
        model_name = self.openai_model_name if provider == LLMProvider.OPENAI else self.anthropic_model_name

        start = time.time()
        result = llm.invoke(msgs)
        latency = time.time() - start

        usage = result.usage_metadata or {}
        inp = usage.get("input_tokens", 0)
        out = usage.get("output_tokens", 0)
        cost = self._estimate_cost(model_name, inp, out)
        self.total_cost += cost
        self.call_count += 1
        self.call_log.append({"model": model_name, "input_tokens": inp,
                              "output_tokens": out, "latency": latency, "cost": cost})
        return LLMResponse(
            content=result.content, provider=provider.value, model=model_name,
            input_tokens=inp, output_tokens=out, total_tokens=inp + out,
            latency_seconds=round(latency, 3), cost_estimate=cost,
        )

    def get_cost_summary(self) -> Dict[str, Any]:
        if not self.call_log:
            return {"total_calls": 0, "total_cost": 0.0}
        df = pd.DataFrame(self.call_log)
        return {
            "total_calls": self.call_count,
            "total_cost_usd": round(self.total_cost, 6),
            "avg_latency_s": round(df["latency"].mean(), 3),
            "avg_tokens": int(df["input_tokens"].mean() + df["output_tokens"].mean()),
            "by_model": df.groupby("model")["cost"].sum().to_dict(),
        }


llm_service = LLMService(openai_model="gpt-4o-mini", anthropic_model="claude-3-5-haiku-latest",
                          temperature=0.7, max_tokens=768)

# Smoke test
_r = llm_service.call("You are a helpful assistant.", "Say hello in one sentence.", LLMProvider.OPENAI)
print(f"✅ LLM Service ready — test: {_r.content[:80]}")

## 3. Agent Definitions and Task Model

Each agent has a **specialized role** with a dedicated system prompt. Tasks are described in natural language and routed to the most suitable agent by the RL coordinator.

In [ ]:
class AgentRole(Enum):
    RESEARCHER = "researcher"
    ANALYST = "analyst"
    CODER = "coder"
    VALIDATOR = "validator"


# ── System prompts per role ──────────────────────────────────────────────────
AGENT_SYSTEM_PROMPTS: Dict[AgentRole, str] = {
    AgentRole.RESEARCHER: (
        "You are a meticulous research specialist. "
        "Given a topic or question, provide well-structured research findings with key facts, "
        "relevant context, and references where possible. "
        "Focus on accuracy and breadth of coverage."
    ),
    AgentRole.ANALYST: (
        "You are an expert data and systems analyst. "
        "Given information or a question, provide structured analysis including pros/cons, "
        "trade-offs, patterns, and data-driven insights. "
        "Use clear reasoning and quantitative arguments where possible."
    ),
    AgentRole.CODER: (
        "You are a senior software engineer. "
        "Given a programming task, produce clean, well-documented code with explanations. "
        "Include error handling, type hints, and follow best practices."
    ),
    AgentRole.VALIDATOR: (
        "You are a quality assurance specialist. "
        "Given a piece of work (research, analysis, or code), verify its correctness, "
        "identify errors or gaps, and provide an improvement checklist. "
        "Be thorough and constructive."
    ),
}


@dataclass
class Task:
    """A task to be processed by the multi-agent system."""
    id: str
    description: str
    task_type: str           # research | analysis | coding | validation
    complexity: float        # 0-1
    priority: float          # 0-1


@dataclass
class AgentState:
    """Tracks an individual agent's state."""
    id: str
    role: AgentRole
    provider: LLMProvider
    completed_tasks: int = 0
    total_reward: float = 0.0
    avg_quality: float = 0.0


# ── Instantiate agent pool ───────────────────────────────────────────────────
AGENTS: Dict[str, AgentState] = {
    "researcher": AgentState("researcher", AgentRole.RESEARCHER, LLMProvider.OPENAI),
    "analyst":    AgentState("analyst",    AgentRole.ANALYST,    LLMProvider.OPENAI),
    "coder":      AgentState("coder",      AgentRole.CODER,      LLMProvider.ANTHROPIC),
    "validator":  AgentState("validator",  AgentRole.VALIDATOR,  LLMProvider.OPENAI),
}

# ── Task bank ────────────────────────────────────────────────────────────────
TASK_BANK: List[Task] = [
    # Research tasks
    Task("r1", "Research the current state of quantum computing and its practical applications.", "research", 0.7, 0.8),
    Task("r2", "Summarize the key differences between supervised, unsupervised, and reinforcement learning.", "research", 0.5, 0.6),
    Task("r3", "What are the main approaches to federated learning and their trade-offs?", "research", 0.6, 0.7),
    # Analysis tasks
    Task("a1", "Analyze the trade-offs between microservices and monolithic architecture for a startup.", "analysis", 0.6, 0.7),
    Task("a2", "Compare REST, GraphQL, and gRPC for inter-service communication.", "analysis", 0.5, 0.8),
    Task("a3", "Evaluate the pros and cons of event-driven vs request-driven architectures.", "analysis", 0.7, 0.6),
    # Coding tasks
    Task("c1", "Write a Python class that implements a thread-safe LRU cache.", "coding", 0.6, 0.9),
    Task("c2", "Implement a Python decorator that retries a function with exponential backoff.", "coding", 0.5, 0.7),
    Task("c3", "Write a Python async producer-consumer pattern using asyncio queues.", "coding", 0.7, 0.8),
    # Validation tasks
    Task("v1", "Review and validate the correctness of this claim: 'TCP guarantees in-order delivery while UDP does not.'", "validation", 0.3, 0.5),
    Task("v2", "Validate the statement: 'B-trees are always more efficient than hash indexes for database lookups.'", "validation", 0.4, 0.6),
]

print(f"✅ Agent pool: {len(AGENTS)} agents")
for aid, a in AGENTS.items():
    print(f"   {a.role.value:12s} → {a.provider.value}")
print(f"✅ Task bank:  {len(TASK_BANK)} tasks")

## 4. LLM-as-Judge Reward Model

A separate LLM call evaluates the quality of agent outputs, producing a scalar reward for the RL coordinator.

In [ ]:
class QualityScores(BaseModel):
    relevance: float = Field(ge=0, le=1, description="How relevant the output is to the task")
    accuracy: float = Field(ge=0, le=1, description="Factual / logical accuracy")
    completeness: float = Field(ge=0, le=1, description="How thoroughly the task is addressed")
    agent_match: float = Field(ge=0, le=1, description="How well the assigned agent role fits the task")
    reasoning: str = Field(description="Brief justification")


JUDGE_SYSTEM = """You are an expert evaluator of AI agent outputs.
You will receive:
- The original task description and type
- The agent role that processed it
- The agent's output

Score the output on each dimension from 0.0 to 1.0.

Return ONLY valid JSON:
{
  "relevance": <float>,
  "accuracy": <float>,
  "completeness": <float>,
  "agent_match": <float>,
  "reasoning": "<brief explanation>"
}

Be critical. 1.0 means perfect."""


class LLMJudge:
    def __init__(self, llm_service: LLMService, provider: LLMProvider = LLMProvider.OPENAI):
        self.llm_service = llm_service
        self.provider = provider
        self.log: List[Dict[str, Any]] = []

    def evaluate(self, task_desc: str, task_type: str, agent_role: str, output: str) -> QualityScores:
        msg = (
            f"## Task\n{task_desc}\n\n"
            f"## Task Type\n{task_type}\n\n"
            f"## Agent Role\n{agent_role}\n\n"
            f"## Agent Output\n{output[:2000]}\n\n"
            f"Evaluate now."
        )
        resp = self.llm_service.call(JUDGE_SYSTEM, msg, self.provider)
        try:
            raw = resp.content.strip()
            if raw.startswith("```"):
                raw = raw.split("```")[1]
                if raw.startswith("json"):
                    raw = raw[4:]
            scores = QualityScores(**json.loads(raw))
        except Exception:
            scores = QualityScores(relevance=0.5, accuracy=0.5, completeness=0.5,
                                   agent_match=0.5, reasoning="Parse error; defaults used.")
        self.log.append({
            "task_type": task_type, "agent_role": agent_role,
            "relevance": scores.relevance, "accuracy": scores.accuracy,
            "completeness": scores.completeness, "agent_match": scores.agent_match,
        })
        return scores

    @staticmethod
    def composite_reward(scores: QualityScores, cost: float, latency: float,
                         cost_w: float = 0.10, latency_w: float = 0.05) -> float:
        quality = np.mean([scores.relevance, scores.accuracy,
                           scores.completeness, scores.agent_match])
        norm_cost = min(cost / 0.002, 1.0)
        norm_lat = min(latency / 5.0, 1.0)
        return float(np.clip(quality - cost_w * norm_cost - latency_w * norm_lat, 0, 1))


judge = LLMJudge(llm_service)
print("✅ LLM-as-Judge ready")

## 5. Q-Learning RL Coordinator

The coordinator learns which **agent** (role + provider) should handle each **task type** to maximise quality while minimising cost and latency.

In [ ]:
class TaskAssignment:
    """Represents an RL action: which agent (role) and provider to assign a task to."""
    def __init__(self, agent_id: str, role: AgentRole, provider: LLMProvider):
        self.agent_id = agent_id
        self.role = role
        self.provider = provider

    def key(self) -> str:
        return f"{self.agent_id}_{self.provider.value}"


# All possible agent-provider assignments
ASSIGNMENT_ACTIONS: List[TaskAssignment] = []
for aid, agent in AGENTS.items():
    for prov in LLMProvider:
        ASSIGNMENT_ACTIONS.append(TaskAssignment(aid, agent.role, prov))

TASK_TYPES = sorted(set(t.task_type for t in TASK_BANK))

print(f"State space (task types): {TASK_TYPES}")
print(f"Action space ({len(ASSIGNMENT_ACTIONS)} assignments):")
for a in ASSIGNMENT_ACTIONS:
    print(f"  {a.agent_id:12s} via {a.provider.value}")


class QLearningCoordinator:
    """Q-Learning agent for task -> agent+provider routing."""

    def __init__(self, task_types: List[str], actions: List[TaskAssignment],
                 lr: float = 0.15, gamma: float = 0.95, epsilon: float = 1.0,
                 epsilon_min: float = 0.05, epsilon_decay: float = 0.97):
        self.task_types = task_types
        self.actions = actions
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

        # Q-table: task_type -> action_idx -> Q-value
        self.q_table: Dict[str, np.ndarray] = {
            tt: np.zeros(len(actions)) for tt in task_types
        }
        self.history: List[Dict[str, Any]] = []

    def select_action(self, task_type: str) -> Tuple[int, TaskAssignment]:
        if np.random.random() < self.epsilon:
            idx = np.random.randint(len(self.actions))
        else:
            idx = int(np.argmax(self.q_table[task_type]))
        return idx, self.actions[idx]

    def update(self, task_type: str, action_idx: int, reward: float,
               next_task_type: Optional[str] = None):
        old_q = self.q_table[task_type][action_idx]
        if next_task_type is not None:
            max_next = np.max(self.q_table[next_task_type])
        else:
            max_next = 0.0
        td_target = reward + self.gamma * max_next
        self.q_table[task_type][action_idx] += self.lr * (td_target - old_q)

        self.history.append({
            "task_type": task_type,
            "action_idx": action_idx,
            "agent": self.actions[action_idx].agent_id,
            "provider": self.actions[action_idx].provider.value,
            "reward": reward,
            "q_value": self.q_table[task_type][action_idx],
        })

    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


coordinator = QLearningCoordinator(TASK_TYPES, ASSIGNMENT_ACTIONS)
print(f"\n✅ Q-Learning Coordinator: {len(TASK_TYPES)} states x {len(ASSIGNMENT_ACTIONS)} actions")

## 6. LangGraph Multi-Agent Workflow

The training loop is orchestrated as a **LangGraph state machine** with five nodes:

```
START --> pick_task --> assign_agent --> agent_execute --> judge_output --> rl_update --> (loop or END)
```

In [ ]:
class WorkflowState(TypedDict):
    """LangGraph state for the multi-agent RL training loop."""
    iteration: int
    max_iterations: int
    task: Optional[Dict[str, Any]]
    assignment: Optional[Dict[str, Any]]
    action_idx: int
    agent_output: str
    llm_response_meta: Optional[Dict[str, Any]]
    scores: Optional[Dict[str, float]]
    reward: float
    # Accumulated metrics
    rewards_log: List[float]
    scores_log: List[Dict[str, Any]]
    assignments_log: List[Dict[str, Any]]


# ── Node functions ────────────────────────────────────────────────────────────

def pick_task(state: WorkflowState) -> dict:
    """Randomly sample a task from the task bank."""
    task = random.choice(TASK_BANK)
    return {
        "task": {
            "id": task.id, "description": task.description,
            "task_type": task.task_type, "complexity": task.complexity,
            "priority": task.priority,
        }
    }


def assign_agent(state: WorkflowState) -> dict:
    """Use the Q-Learning coordinator to choose an agent + provider."""
    task_type = state["task"]["task_type"]
    action_idx, assignment = coordinator.select_action(task_type)
    return {
        "assignment": {
            "agent_id": assignment.agent_id,
            "role": assignment.role.value,
            "provider": assignment.provider.value,
        },
        "action_idx": action_idx,
    }


def agent_execute(state: WorkflowState) -> dict:
    """Execute the task with a real LLM call through the assigned agent."""
    task = state["task"]
    assign = state["assignment"]

    role = AgentRole(assign["role"])
    provider = LLMProvider(assign["provider"])

    system_prompt = AGENT_SYSTEM_PROMPTS[role]
    user_message = (
        f"[{task['task_type'].upper()} TASK | complexity={task['complexity']:.1f} | "
        f"priority={task['priority']:.1f}]\n\n{task['description']}"
    )

    response = llm_service.call(system_prompt, user_message, provider)

    return {
        "agent_output": response.content,
        "llm_response_meta": {
            "provider": response.provider,
            "model": response.model,
            "tokens": response.total_tokens,
            "latency": response.latency_seconds,
            "cost": response.cost_estimate,
        },
    }


def judge_output(state: WorkflowState) -> dict:
    """Use the LLM-as-Judge to score the agent output."""
    task = state["task"]
    assign = state["assignment"]
    scores = judge.evaluate(
        task_desc=task["description"],
        task_type=task["task_type"],
        agent_role=assign["role"],
        output=state["agent_output"],
    )
    score_dict = {
        "relevance": scores.relevance,
        "accuracy": scores.accuracy,
        "completeness": scores.completeness,
        "agent_match": scores.agent_match,
        "reasoning": scores.reasoning,
    }
    meta = state["llm_response_meta"]
    reward = LLMJudge.composite_reward(scores, meta["cost"], meta["latency"])
    return {"scores": score_dict, "reward": reward}


def rl_update(state: WorkflowState) -> dict:
    """Update Q-table and log metrics."""
    task_type = state["task"]["task_type"]
    action_idx = state["action_idx"]
    reward = state["reward"]

    # Peek at next task type for bootstrapping (random)
    next_type = random.choice(TASK_TYPES)
    coordinator.update(task_type, action_idx, reward, next_type)
    coordinator.decay_epsilon()

    # Update agent-level stats
    aid = state["assignment"]["agent_id"]
    ag = AGENTS[aid]
    ag.completed_tasks += 1
    ag.total_reward += reward
    ag.avg_quality = ag.total_reward / ag.completed_tasks

    # Build new lists (idiomatic LangGraph pattern — return new values, never mutate)
    prev_rewards = state.get("rewards_log", []) or []
    prev_scores = state.get("scores_log", []) or []
    prev_assignments = state.get("assignments_log", []) or []

    it = state["iteration"] + 1
    print(f"   [{it}/{state['max_iterations']}] reward={reward:.3f} "
          f"agent={aid} provider={state['assignment']['provider']}")

    return {
        "rewards_log": prev_rewards + [reward],
        "scores_log": prev_scores + [state["scores"]],
        "assignments_log": prev_assignments + [{
            "iteration": state["iteration"],
            "task_type": task_type,
            "agent": aid,
            "provider": state["assignment"]["provider"],
            "reward": reward,
            **{k: v for k, v in state["scores"].items() if k != "reasoning"},
        }],
        "iteration": it,
    }


def should_continue(state: WorkflowState) -> str:
    if state["iteration"] >= state["max_iterations"]:
        return END
    return "pick_task"


# ── Build the graph ──────────────────────────────────────────────────────────
workflow = StateGraph(WorkflowState)

workflow.add_node("pick_task", pick_task)
workflow.add_node("assign_agent", assign_agent)
workflow.add_node("agent_execute", agent_execute)
workflow.add_node("judge_output", judge_output)
workflow.add_node("rl_update", rl_update)

workflow.add_edge(START, "pick_task")
workflow.add_edge("pick_task", "assign_agent")
workflow.add_edge("assign_agent", "agent_execute")
workflow.add_edge("agent_execute", "judge_output")
workflow.add_edge("judge_output", "rl_update")
workflow.add_conditional_edges("rl_update", should_continue, {END: END, "pick_task": "pick_task"})

app = workflow.compile()
print("✅ LangGraph workflow compiled")

## 7. Training Run

Each iteration makes 2 LLM calls (agent + judge). With 30 iterations that is approximately 60 API calls.

Estimated cost: less than $0.10 with gpt-4o-mini / claude-3-5-haiku.

In [ ]:
N_ITERATIONS = 30  # Increase for better convergence (at higher cost)

initial_state: WorkflowState = {
    "iteration": 0,
    "max_iterations": N_ITERATIONS,
    "task": None,
    "assignment": None,
    "action_idx": 0,
    "agent_output": "",
    "llm_response_meta": None,
    "scores": None,
    "reward": 0.0,
    "rewards_log": [],
    "scores_log": [],
    "assignments_log": [],
}

print(f"Starting training for {N_ITERATIONS} iterations...")
print(f"Initial epsilon: {coordinator.epsilon:.2f}\n")

start_time = time.time()
final_state = app.invoke(initial_state)
elapsed = time.time() - start_time

print(f"\n{'='*60}")
print(f"Training complete in {elapsed:.1f}s")
print(f"  Iterations:        {final_state['iteration']}")
print(f"  Mean reward:       {np.mean(final_state['rewards_log']):.3f}")
print(f"  Final epsilon:     {coordinator.epsilon:.4f}")
print(f"  Total LLM calls:   {llm_service.call_count}")
print(f"  Total cost (USD):  ${llm_service.total_cost:.4f}")
print(f"{'='*60}")

## 8. Training Visualization

In [ ]:
df_log = pd.DataFrame(final_state["assignments_log"])

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle("Multi-Agent RL Training Dashboard", fontsize=16, fontweight="bold")

# ── 1. Reward curve ──────────────────────────────────────────────────────────
ax = axes[0, 0]
rewards = final_state["rewards_log"]
ax.plot(rewards, "o-", alpha=0.5, markersize=4, label="Per-iteration")
window = min(5, len(rewards))
if len(rewards) >= window:
    rolling = pd.Series(rewards).rolling(window).mean()
    ax.plot(rolling, "r-", linewidth=2, label=f"{window}-step MA")
ax.set_xlabel("Iteration")
ax.set_ylabel("Reward")
ax.set_title("Reward Over Training")
ax.legend()
ax.grid(True, alpha=0.3)

# ── 2. Quality score breakdown ───────────────────────────────────────────────
ax = axes[0, 1]
score_cols = ["relevance", "accuracy", "completeness", "agent_match"]
for col in score_cols:
    ax.plot(df_log[col], "o-", markersize=3, alpha=0.6, label=col)
ax.set_xlabel("Iteration")
ax.set_ylabel("Score")
ax.set_title("Judge Quality Scores")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# ── 3. Agent utilisation ─────────────────────────────────────────────────────
ax = axes[1, 0]
usage = df_log.groupby(["agent", "provider"]).size().reset_index(name="count")
bars = [f"{r.agent}\n({r.provider})" for _, r in usage.iterrows()]
colors = plt.cm.Set2(np.linspace(0, 1, len(bars)))
ax.bar(bars, usage["count"], color=colors)
ax.set_title("Agent-Provider Utilisation")
ax.set_ylabel("Tasks Assigned")

# ── 4. Avg reward by task type ───────────────────────────────────────────────
ax = axes[1, 1]
avg_by_type = df_log.groupby("task_type")["reward"].mean().sort_values()
avg_by_type.plot(kind="barh", ax=ax, color="steelblue")
ax.set_xlabel("Mean Reward")
ax.set_title("Mean Reward by Task Type")

plt.tight_layout()
plt.show()

## 9. Learned Q-Table Policy

In [ ]:
# ── Q-table heatmap ──────────────────────────────────────────────────────────
action_labels = [f"{a.agent_id}\n({a.provider.value[:3]})" for a in ASSIGNMENT_ACTIONS]
q_matrix = np.array([coordinator.q_table[tt] for tt in TASK_TYPES])

fig, ax = plt.subplots(figsize=(14, 4))
sns.heatmap(q_matrix, annot=True, fmt=".2f", cmap="YlGnBu",
            xticklabels=action_labels, yticklabels=TASK_TYPES, ax=ax)
ax.set_title("Learned Q-Table  (task type x agent-provider)")
ax.set_xlabel("Agent (Provider)")
ax.set_ylabel("Task Type")
plt.tight_layout()
plt.show()

# ── Best assignment per task type ────────────────────────────────────────────
print("\nLearned Routing Policy:")
print(f"{'Task Type':<14} {'Best Agent':<14} {'Provider':<12} {'Q-value':>8}")
print("-" * 52)
for tt in TASK_TYPES:
    best_idx = int(np.argmax(coordinator.q_table[tt]))
    best = ASSIGNMENT_ACTIONS[best_idx]
    q_val = coordinator.q_table[tt][best_idx]
    print(f"{tt:<14} {best.agent_id:<14} {best.provider.value:<12} {q_val:>8.3f}")

## 10. Live Demo: Using the Trained Policy

Route a new, unseen task through the learned policy and show the full pipeline output.

In [ ]:
def demo_task(description: str, task_type: str, verbose: bool = True) -> Dict[str, Any]:
    """Run a single task through the trained policy (greedy, no exploration)."""
    old_eps = coordinator.epsilon
    coordinator.epsilon = 0.0  # greedy

    action_idx, assignment = coordinator.select_action(task_type)
    role = assignment.role
    provider = assignment.provider

    if verbose:
        print(f"Task:     {description}")
        print(f"Type:     {task_type}")
        print(f"Routed -> {assignment.agent_id} ({role.value}) via {provider.value}")

    response = llm_service.call(
        AGENT_SYSTEM_PROMPTS[role],
        f"[{task_type.upper()} TASK]\n\n{description}",
        provider,
    )
    scores = judge.evaluate(description, task_type, role.value, response.content)
    reward = LLMJudge.composite_reward(scores, response.cost_estimate, response.latency_seconds)

    if verbose:
        print(f"\n--- Agent Output (first 500 chars) ---")
        print(response.content[:500])
        print(f"\n--- Judge Scores ---")
        print(f"  Relevance:     {scores.relevance:.2f}")
        print(f"  Accuracy:      {scores.accuracy:.2f}")
        print(f"  Completeness:  {scores.completeness:.2f}")
        print(f"  Agent Match:   {scores.agent_match:.2f}")
        print(f"  Reward:        {reward:.3f}")
        print(f"  Reasoning:     {scores.reasoning}")
        print(f"\n  Model: {response.model} | Tokens: {response.total_tokens} | "
              f"Latency: {response.latency_seconds:.2f}s | Cost: ${response.cost_estimate:.5f}")

    coordinator.epsilon = old_eps
    return {"response": response.content, "scores": scores, "reward": reward}


# ── Try a fresh task ─────────────────────────────────────────────────────────
print("=" * 70)
print("LIVE DEMO")
print("=" * 70)
demo_task(
    "Explain the CAP theorem and give real-world examples of systems that "
    "trade off each guarantee.",
    task_type="research",
)

## 11. Production Integration

A self-contained class that wraps the trained RL coordinator for use in a production service. Supports online learning by feeding live judge scores back into the Q-table.

In [ ]:
class ProductionMultiAgentSystem:
    """Wraps the trained RL coordinator + real LLM agents for production use."""

    def __init__(self, coordinator: QLearningCoordinator, llm_service: LLMService,
                 judge: LLMJudge, agents: Dict[str, AgentState],
                 online_learning: bool = True):
        self.coordinator = coordinator
        self.llm_service = llm_service
        self.judge = judge
        self.agents = agents
        self.online_learning = online_learning
        # Rate limiting
        self.request_times: deque = deque(maxlen=200)
        self.max_rpm: int = 60
        self.request_log: List[Dict[str, Any]] = []

    def _check_rate_limit(self) -> bool:
        now = time.time()
        self.request_times.append(now)
        cutoff = now - 60
        recent = sum(1 for t in self.request_times if t > cutoff)
        return recent <= self.max_rpm

    def process(self, description: str, task_type: str) -> Dict[str, Any]:
        """Route a task, execute, evaluate, optionally learn."""
        if not self._check_rate_limit():
            return {"error": "Rate limit exceeded"}

        # 1. Route
        self.coordinator.epsilon = 0.0  # greedy in production
        action_idx, assignment = self.coordinator.select_action(task_type)
        role = assignment.role
        provider = assignment.provider

        # 2. Execute
        response = self.llm_service.call(
            AGENT_SYSTEM_PROMPTS[role],
            f"[{task_type.upper()} TASK]\n\n{description}",
            provider,
        )

        # 3. Judge
        scores = self.judge.evaluate(description, task_type, role.value, response.content)
        reward = LLMJudge.composite_reward(scores, response.cost_estimate, response.latency_seconds)

        # 4. Online learning
        if self.online_learning:
            self.coordinator.update(task_type, action_idx, reward)

        result = {
            "agent": assignment.agent_id,
            "provider": provider.value,
            "model": response.model,
            "response": response.content,
            "tokens": response.total_tokens,
            "latency": response.latency_seconds,
            "cost": response.cost_estimate,
            "reward": reward,
            "scores": {
                "relevance": scores.relevance,
                "accuracy": scores.accuracy,
                "completeness": scores.completeness,
                "agent_match": scores.agent_match,
            },
        }
        self.request_log.append(result)
        return result

    def health(self) -> Dict[str, Any]:
        return {
            "status": "healthy",
            "total_requests": len(self.request_log),
            "total_llm_calls": self.llm_service.call_count,
            "total_cost_usd": round(self.llm_service.total_cost, 5),
            "avg_reward": round(np.mean([r["reward"] for r in self.request_log]), 3) if self.request_log else 0.0,
        }


# ── Instantiate production system ────────────────────────────────────────────
prod = ProductionMultiAgentSystem(coordinator, llm_service, judge, AGENTS)

# Quick test
result = prod.process(
    "Write a Python function that merges two sorted linked lists into one sorted list.",
    "coding",
)
print("Production system test:")
print(f"  Agent:    {result['agent']} via {result['provider']}")
print(f"  Tokens:   {result['tokens']}")
print(f"  Reward:   {result['reward']:.3f}")
print(f"  Output:   {result['response'][:200]}...")
print(f"\nHealth: {json.dumps(prod.health(), indent=2)}")

## 12. Cost and Agent Summary

In [ ]:
# ── Cost breakdown ────────────────────────────────────────────────────────────
cost_summary = llm_service.get_cost_summary()
print("LLM Cost Summary")
print("=" * 40)
for k, v in cost_summary.items():
    print(f"  {k:20s} {v}")

# ── Agent performance ─────────────────────────────────────────────────────────
print("\nAgent Performance Summary")
print(f"{'Agent':<14} {'Tasks':>6} {'Avg Reward':>11} {'Total Reward':>13}")
print("-" * 48)
for aid, ag in AGENTS.items():
    print(f"{aid:<14} {ag.completed_tasks:>6} {ag.avg_quality:>11.3f} {ag.total_reward:>13.3f}")

## Summary

This notebook replaced the **simulated multi-agent environment** from the original notebook with **real LLM API calls** orchestrated by **LangGraph**:

| Component | Implementation |
|---|---|
| **Agent execution** | Real LangChain calls to OpenAI (gpt-4o-mini) and Anthropic (claude-3-5-haiku) |
| **Task routing** | Q-Learning coordinator learns optimal task-type to agent+provider mapping |
| **Quality evaluation** | LLM-as-Judge scores outputs on relevance, accuracy, completeness, and agent-match |
| **Workflow orchestration** | LangGraph state machine: pick_task, assign_agent, agent_execute, judge_output, rl_update |
| **Cost control** | Token tracking and USD cost estimation per call; configurable iteration count |
| **Production readiness** | ProductionMultiAgentSystem class with rate limiting and online learning |

### Key insights
- The RL coordinator learns which **agent role + LLM provider** combination yields the highest quality for each task type.
- The **LLM-as-Judge** provides consistent, multi-dimensional evaluation without human labelling.
- **LangGraph** cleanly separates concerns into nodes, making the pipeline easy to extend (e.g., add a reviewer agent, human-in-the-loop, or tool-augmented agents).
- Running with budget-friendly models (gpt-4o-mini, claude-3-5-haiku) keeps total training cost minimal while demonstrating the full workflow.